装饰器的一些例子

https://wiki.python.org/moin/PythonDecoratorLibrary#Property_Definition

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## creating well-behaved decorators / "Decorator decorator"

In [ ]:
def simple_decorator(decorator):
    '''This decorator can be used to turn simple functions
    into well-behaved decorators, so long as the decorators
    are fairly simple. If a decorator expects a function and
    returns a function (no descriptors), and if it doesn't
    modify function attributes or docstring, then it is
    eligible to use this. Simply apply @simple_decorator to
    your decorator and it will automatically preserve the
    docstring and function attributes of functions to which
    it is applied.'''
    def new_decorator(f):
        g = decorator(f)
        g.__name__ = f.__name__
        g.__doc__ = f.__doc__
        g.__dict__.update(f.__dict__)
        return g
    # Now a few lines needed to make simple_decorator itself
    # be a well-behaved decorator.
    new_decorator.__name__ = decorator.__name__
    new_decorator.__doc__ = decorator.__doc__
    new_decorator.__dict__.update(decorator.__dict__)
    return new_decorator


#
# Sample Use:
#
@simple_decorator
def my_simple_logging_decorator(func):
    def you_will_never_see_this_name(*args, **kwargs):
        print ('calling {}'.format(func.__name__))
        return func(*args, **kwargs)
    return you_will_never_see_this_name

@my_simple_logging_decorator
def double(x):
    'Doubles a number.'
    return 2 * x

assert double.__name__ == 'double'
assert double.__doc__ == 'Doubles a number.'
print (double(155))

In [ ]:
def simple_decorator(decorator):
    '''This decorator can be used to turn simple functions
    into well-behaved decorators, so long as the decorators
    are fairly simple. If a decorator expects a function and
    returns a function (no descriptors), and if it doesn't
    modify function attributes or docstring, then it is
    eligible to use this. Simply apply @simple_decorator to
    your decorator and it will automatically preserve the
    docstring and function attributes of functions to which
    it is applied.'''
    def new_decorator(f):
        g = decorator(f)
        g.__name__ = f.__name__
        g.__doc__ = f.__doc__
        g.__dict__.update(f.__dict__)
        return g
    # Now a few lines needed to make simple_decorator itself
    # be a well-behaved decorator.
    new_decorator.__name__ = decorator.__name__
    new_decorator.__doc__ = decorator.__doc__
    new_decorator.__dict__.update(decorator.__dict__)
    return new_decorator


#
# Sample Use:
#
#@simple_decorator
def my_simple_logging_decorator(func):
    def you_will_never_see_this_name(*args, **kwargs):
        print ('calling {}'.format(func.__name__))
        return func(*args, **kwargs)
    return you_will_never_see_this_name

@my_simple_logging_decorator
def double(x):
    'Doubles a number.'
    return 2 * x

print(double.__name__)
print(double.__doc__)
print (double(155))

assert double.__name__ == 'double'
assert double.__doc__ == 'Doubles a number.'

In [ ]:
double(155)
print(double.__name__)
print(double.__doc__)

In [ ]:
print(double.__name__)

In [ ]:
print(double.__doc__)

In [ ]:
def double(x):
    'Doubles a number.'
    return 2 * x

In [ ]:
double.__name__

In [ ]:
double.__doc__

## prpoerty definition

In [ ]:
property??

In [ ]:
import sys

def propget(func):
    locals = sys._getframe(1).f_locals
    name = func.__name__
    prop = locals.get(name)
    if not isinstance(prop, property):
        prop = property(func, doc=func.__doc__)
    else:
        doc = prop.__doc__ or func.__doc__
        prop = property(func, prop.fset, prop.fdel, doc)
    return prop

def propset(func):
    locals = sys._getframe(1).f_locals
    name = func.__name__
    prop = locals.get(name)
    if not isinstance(prop, property):
        prop = property(None, func, doc=func.__doc__)
    else:
        doc = prop.__doc__ or func.__doc__
        prop = property(prop.fget, func, prop.fdel, doc)
    return prop

def propdel(func):
    locals = sys._getframe(1).f_locals
    name = func.__name__
    prop = locals.get(name)
    if not isinstance(prop, property):
        prop = property(None, None, func, doc=func.__doc__)
    else:
        prop = property(prop.fget, prop.fset, func, prop.__doc__)
    return prop

# These can be used like this:

class Example(object):

    @propget
    def myattr(self):
        return self._half * 2

    @propset
    def myattr(self, value):
        self._half = value / 2

    @propdel
    def myattr(self):
        del self._half

### 另一个例子

In [ ]:
import sys

try:
    # Python 2
    import __builtin__ as builtins
except ImportError:
    # Python 3
    import builtins

def property(function):
    keys = 'fget', 'fset', 'fdel'
    func_locals = {'doc':function.__doc__}
    def probe_func(frame, event, arg):
        if event == 'return':
            locals = frame.f_locals
            func_locals.update(dict((k, locals.get(k)) for k in keys))
            sys.settrace(None)
        return probe_func
    sys.settrace(probe_func)
    function()
    return builtins.property(**func_locals)

#====== Example =======================================================

from math import radians, degrees, pi

class Angle(object):
    def __init__(self, rad):
        self._rad = rad

    @property
    def rad():
        '''The angle in radians'''
        def fget(self):
            return self._rad
        def fset(self, angle):
            if isinstance(angle, Angle):
                angle = angle.rad
            self._rad = float(angle)

    @property
    def deg():
        '''The angle in degrees'''
        def fget(self):
            return degrees(self._rad)
        def fset(self, angle):
            if isinstance(angle, Angle):
                angle = angle.deg
            self._rad = radians(angle)

In [ ]:
sys.settrace??

## 关于函数记忆的

In [1]:
import collections
import functools

这是一个具有记忆功能的类

In [ ]:
class memoized(object):
    """Decorator. Caches a function's return value each time it is called.
   If called later with the same arguments, the cached value is returned
   (not reevaluated).
   """ 

    def __init__(self, func):
        self.func = func
        self.cache = {}   
    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
             # uncacheable. a list, for instance.
             # better to not cache than blow up.
             return self.func(*args)
        if args in self.cache:
             return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            return value
    def __repr__(self):
        '''Return the function's docstring.'''
        return self.func.__doc__
    def __get__(self, obj, objtype):
        '''Support instance methods.'''
        return functools.partial(self.__call__, obj)

In [ ]:
@memoized
def fibonacci(n):
    "Return the nth fibonacci number."
    if n in (0, 1):
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [ ]:
print(fibonacci(15))

In [ ]:
fibonacci(12)
fibonacci(10)
fibonacci(7)

In [ ]:
fibonacci.cache

这是一个具有记忆功能的函数，它的缓存是暴露在外面的。

In [8]:
# note that this decorator ignores **kwargs
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        if args not in cache:
            cache[args] = obj(*args, **kwargs)
        return cache[args]
    return memoizer

In [9]:
@memoize
def fibonacci(n):
    "Return the nth fibonacci number."
    if n in (0, 1):
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [10]:
fibonacci(20)

6765

In [11]:
fibonacci.cache

{(1,): 1,
 (0,): 0,
 (2,): 1,
 (3,): 2,
 (4,): 3,
 (5,): 5,
 (6,): 8,
 (7,): 13,
 (8,): 21,
 (9,): 34,
 (10,): 55,
 (11,): 89,
 (12,): 144,
 (13,): 233,
 (14,): 377,
 (15,): 610,
 (16,): 987,
 (17,): 1597,
 (18,): 2584,
 (19,): 4181,
 (20,): 6765}

又一个改进的版本，
a modified version that also respects kwargs.

In [3]:
def memoize(obj):
    cache = obj.cache = {}

    @functools.wraps(obj)
    def memoizer(*args, **kwargs):
        key = str(args) + str(kwargs)
        #print(key)
        if key not in cache:
            cache[key] = obj(*args, **kwargs)
        return cache[key]
    return memoizer

In [4]:
@memoize
def fibonacci(n):
    "Return the nth fibonacci number."
    if n in (0, 1):
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [6]:
fibonacci(10)

55

In [7]:
fibonacci.cache

{'(1,){}': 1,
 '(0,){}': 0,
 '(2,){}': 1,
 '(3,){}': 2,
 '(4,){}': 3,
 '(5,){}': 5,
 '(6,){}': 8,
 '(7,){}': 13,
 '(8,){}': 21,
 '(9,){}': 34,
 '(10,){}': 55}

作为字典亚类的记忆器。

In [12]:
class memoize(dict):
    def __init__(self, func):
        self.func = func

    def __call__(self, *args):
        return self[args]

    def __missing__(self, key):
        result = self[key] = self.func(*key)
        return result

In [13]:
#
# Sample use
#

@memoize
def foo(a, b):
    return a * b
foo(2, 4)

foo

foo('hi', 3)

foo


8

{(2, 4): 8}

'hihihi'

{(2, 4): 8, ('hi', 3): 'hihihi'}

又一个关于 cache 的例子
Alternate memoize that stores cache between executions

https://github.com/brmscheiner/memorize.py

In [14]:
import pickle
import collections
import functools
import inspect
import os.path
import re
import unicodedata

class Memorize(object):
    '''
    A function decorated with @Memorize caches its return
    value every time it is called. If the function is called
    later with the same arguments, the cached value is
    returned (the function is not reevaluated). The cache is
    stored as a .cache file in the current directory for reuse
    in future executions. If the Python file containing the
    decorated function has been updated since the last run,
    the current cache is deleted and a new cache is created
    (in case the behavior of the function has changed).
    '''
    def __init__(self, func):
        self.func = func
        self.set_parent_file() # Sets self.parent_filepath and self.parent_filename
        self.__name__ = self.func.__name__
        self.set_cache_filename()
        if self.cache_exists():
            self.read_cache() # Sets self.timestamp and self.cache
            if not self.is_safe_cache():
                self.cache = {}
        else:
            self.cache = {}

    def __call__(self, *args):
        if not isinstance(args, collections.Hashable):
            return self.func(*args)
        if args in self.cache:
            return self.cache[args]
        else:
            value = self.func(*args)
            self.cache[args] = value
            self.save_cache()
            return value

    def set_parent_file(self):
        """
        Sets self.parent_file to the absolute path of the
        file containing the memoized function.
        """
        rel_parent_file = inspect.stack()[-1].filename
        self.parent_filepath = os.path.abspath(rel_parent_file)
        self.parent_filename = _filename_from_path(rel_parent_file)

    def set_cache_filename(self):
        """
        Sets self.cache_filename to an os-compliant
        version of "file_function.cache"
        """
        filename = _slugify(self.parent_filename.replace('.py', ''))
        funcname = _slugify(self.__name__)
        self.cache_filename = filename+'_'+funcname+'.cache'

    def get_last_update(self):
        """
        Returns the time that the parent file was last
        updated.
        """
        last_update = os.path.getmtime(self.parent_filepath)
        return last_update

    def is_safe_cache(self):
        """
        Returns True if the file containing the memoized
        function has not been updated since the cache was
        last saved.
        """
        if self.get_last_update() > self.timestamp:
            return False
        return True

    def read_cache(self):
        """
        Read a pickled dictionary into self.timestamp and
        self.cache. See self.save_cache.
        """
        with open(self.cache_filename, 'rb') as f:
            data = pickle.loads(f.read())
            self.timestamp = data['timestamp']
            self.cache = data['cache']

    def save_cache(self):
        """
        Pickle the file's timestamp and the function's cache
        in a dictionary object.
        """
        with open(self.cache_filename, 'wb+') as f:
            out = dict()
            out['timestamp'] = self.get_last_update()
            out['cache'] = self.cache
            f.write(pickle.dumps(out))

    def cache_exists(self):
        '''
        Returns True if a matching cache exists in the current directory.
        '''
        if os.path.isfile(self.cache_filename):
            return True
        return False

    def __repr__(self):
        """ Return the function's docstring. """
        return self.func.__doc__

    def __get__(self, obj, objtype):
        """ Support instance methods. """
        return functools.partial(self.__call__, obj)

def _slugify(value):
    """
    Normalizes string, converts to lowercase, removes
    non-alpha characters, and converts spaces to
    hyphens. From
    http://stackoverflow.com/questions/295135/turn-a-string-into-a-valid-filename-in-python
    """
    value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore')
    value = re.sub(r'[^\w\s-]', '', value.decode('utf-8', 'ignore'))
    value = value.strip().lower()
    value = re.sub(r'[-\s]+', '-', value)
    return value

def _filename_from_path(filepath):
    return filepath.split('/')[-1]

## cached properties

In [1]:
#
# © 2011 Christopher Arndt, MIT License
#

import time

class cached_property(object):
    '''Decorator for read-only properties evaluated only once within TTL period.

    It can be used to create a cached property like this::

        import random

        # the class containing the property must be a new-style class
        class MyClass(object):
            # create property whose value is cached for ten minutes
            @cached_property(ttl=600)
            def randint(self):
                # will only be evaluated every 10 min. at maximum.
                return random.randint(0, 100)

    The value is cached  in the '_cache' attribute of the object instance that
    has the property getter method wrapped by this decorator. The '_cache'
    attribute value is a dictionary which has a key for every property of the
    object which is wrapped by this decorator. Each entry in the cache is
    created only when the property is accessed for the first time and is a
    two-element tuple with the last computed property value and the last time
    it was updated in seconds since the epoch.

    The default time-to-live (TTL) is 300 seconds (5 minutes). Set the TTL to
    zero for the cached value to never expire.

    To expire a cached property value manually just do::

        del instance._cache[<property name>]

    '''
    def __init__(self, ttl=300):
        self.ttl = ttl

    def __call__(self, fget, doc=None):
        self.fget = fget
        self.__doc__ = doc or fget.__doc__
        self.__name__ = fget.__name__
        self.__module__ = fget.__module__
        return self

    def __get__(self, inst, owner):
        now = time.time()
        try:
            value, last_update = inst._cache[self.__name__]
            if self.ttl > 0 and now - last_update > self.ttl:
                raise AttributeError
        except (KeyError, AttributeError):
            value = self.fget(inst)
            try:
                cache = inst._cache
            except AttributeError:
                cache = inst._cache = {}
            cache[self.__name__] = (value, now)
        return value

上面那个大概做的事情是，设置了个缓存自动更新的时间。但是怎么用还没看太明白。

In [9]:
import random

# the class containing the property must be a new-style class
class MyClass(object):
    # create property whose value is cached for ten minutes
    @cached_property(ttl=10)
    def randint(self):
        # will only be evaluated every 10 min. at maximum.
        return random.randint(0, 100)

In [10]:
rand = MyClass()

In [28]:
# 把 ttl 改成了 10，意味着上一次的运行可以缓存 10 秒。其他更具体的细节看不太懂了。

rand.randint

8

In [30]:
rand._cache

{'randint': (8, 1634266289.71584)}

In [33]:
rand.__doc__

In [36]:
rand.__name__

AttributeError: 'MyClass' object has no attribute '__name__'

In [37]:
rand.__module__

'__main__'

## Retry

Call a function which returns True/False to indicate success or failure. On failure, wait, and try the function again. On repeated failures, wait longer between each successive attempt. If the decorator runs out of attempts, then it gives up and returns False, but you could just as easily raise some exception.

In [14]:
import time
import math

# Retry decorator with exponential backoff
def retry(tries, delay=3, backoff=2):
    '''Retries a function or method until it returns True.

    delay sets the initial delay in seconds, and backoff sets the factor by which
    the delay should lengthen after each failure. backoff must be greater than 1,
    or else it isn't really a backoff. tries must be at least 0, and delay
    greater than 0.'''
    
    if backoff <= 1:
        raise ValueError("backoff must be greater than 1")
        
    tries = math.floor(tries)
    if tries < 0:
        raise ValueError("tries must be 0 or greater")
        
    if delay <= 0:
        raise ValueError("delay must be greater than 0")
        
    def deco_retry(f):
        def f_retry(*args, **kwargs):
            mtries, mdelay = tries, delay # make mutable
            
            rv = f(*args, **kwargs) # first attempt
            while mtries > 0:
                if rv is True: # Done on success
                    return True
            
                mtries -= 1      # consume an attempt
                time.sleep(mdelay) # wait...
                mdelay *= backoff  # make future wait longer
                
                rv = f(*args, **kwargs) # Try again
                
            return False # Ran out of tries :-(
        
        return f_retry #  true decorator -> decorated function
    return deco_retry  # @retry(arg[, ...]) -> true decorator

例子 设计一个随机数生成器，如果是奇数，返回 True， 如果是偶数，返回 False。

In [24]:
import numpy as np

number = np.random.randint(0, 10000)

@retry(2)
def even_num(num):
    if num % 2 == 0:
        print(num)
        return True
    else:
        print(num)
        return False

even_num(number)

5240


True

In [17]:
even_num(number)

False

例子不太恰当 但确实是这么个意思。

## Pseudo-currying

FYI (for your information), you can use functools.partial() to emulate currying (which works even for keyword arguments).

简单理解，就是当一个函数的参数没给完时，就返回这个函数和已经给出的参数，直到所有的参数都给出了之后才执行这个函数。

In [2]:
class curried(object):
    '''
    Decorator that returns a function that keeps returning functions
    until all arguments are supplied; then the original function is
    evaluated.
    '''

    def __init__(self, func, *a):
        self.func = func
        self.args = a

    def __call__(self, *a):
        args = self.args + a
        if len(args) < self.func.__code__.co_argcount:
            return curried(self.func, *args)
        else:
            return self.func(*args)



In [3]:
@curried
def add(a, b):
    return a + b

add1 = add(1)

In [5]:
add1(2)

3

## 具有可选参数的装饰器

creating decorator with opentional arguments

In [6]:
import functools, inspect

def decorator(func):
    ''' Allow to use decorator either with arguments or not. '''

    def isFuncArg(*args, **kw):
        return len(args) == 1 and len(kw) == 0 and (
            inspect.isfunction(args[0]) or isinstance(args[0], type))

    if isinstance(func, type):
        def class_wrapper(*args, **kw):
            if isFuncArg(*args, **kw):
                return func()(*args, **kw) # create class before usage
            return func(*args, **kw)
        class_wrapper.__name__ = func.__name__
        class_wrapper.__module__ = func.__module__
        return class_wrapper

    @functools.wraps(func)
    def func_wrapper(*args, **kw):
        if isFuncArg(*args, **kw):
            return func(*args, **kw)

        def functor(userFunc):
            return func(userFunc, *args, **kw)

        return functor

    return func_wrapper

没看懂干啥

例子

In [7]:
@decorator
def apply(func, *args, **kw):
    return func(*args, **kw)

In [8]:
@decorator
class apply:
    def __init__(self, *args, **kw):
        self.args = args
        self.kw   = kw

    def __call__(self, func):
        return func(*self.args, **self.kw)

In [21]:
#
# Usage in both cases:
#
@apply
def test():
    return 'test'

In [23]:
assert test == 'test'

In [24]:
def test():
    return 'test'

In [25]:
test()

'test'

这里，我理解的，test应该是个函数，所以调用的时候应该是 test() 这样的形式。可是用了装饰器之后，它的类型似乎就变成了一个字符串。

In [26]:
@apply(2, 3)
def test(a, b):
    return a + b

assert test is 5

In [27]:
@apply(2, 3, 7)
def test(a, b):
    return a + b

TypeError: test() takes 2 positional arguments but 3 were given

在装饰器后面给参数，可以把参数给到函数里面。大概就是干了这么个事情

In [29]:
@apply('leeyy')
def argu_deco(strings):
    return('You give me a {}'.format(strings) )

In [31]:
argu_deco()

TypeError: 'str' object is not callable

In [33]:
argu_deco

'You give me a leeyy'

这个装饰器的缺点

Note: There is only one drawback: wrapper checks its arguments for single function or class. To avoid wrong behavior you can use keyword arguments instead of positional, e.g.:

没看懂

In [34]:
@decorator
def my_property(getter, *, setter=None, deleter=None, doc=None):
    return property(getter, setter, deleter, doc)